In [2]:

import numpy as np
import pandas as pd
import fdata

1) Missing Values:
Catalyst and Float
    
2) Data types:
most are correct, date and time columns need special attention in order to give them the right format.
    
3) Outliers:
some values seem unusual, for example 'Negative RRR in-trade' with a max value of 0.4 whereas the highest value should be 0.
    

In [3]:
dt1 = pd.read_excel(r'Live day trading 2 fixed.xlsx' , 'TABLE1')
dt2 = pd.read_excel(r'Live day trading 4.xlsx' , 'TABLE1')
dt3 = pd.read_excel(r'Live day trading 5.xlsx' , 'TABLE1')
dt4 = pd.read_excel(r'Live day trading 7.xlsx' , 'TABLE1')

Example

1) Sign the tables, and remove their nulls under the "Instrument" column.

2) In dt3 and dt4 which are more recent data tables there has been a separation between 'Potential Price' and 'Potential Price in Trade', the latter is equal to 'Potential Price' in dt2 and d1 which determines the poteintial price a trade may reach without dropping back to SL. Same with 'RRR Potential in Trade' in dt3 and dt4 and 'RRR Potential' in dt2 and dt1.

3) Merge all 4 tables.


In [4]:
#1)

dt_all = [dt1,dt2,dt3,dt4]
table_num = 0

# sign the tables by enumeration
for table in dt_all:

    table["Table Number"] = table_num 
    table = table.dropna(subset=["Instrument"])
    table_num +=1

#2)
dt3 =  dt3.drop(['Potential Price', 'Trade Potential'], axis = 1) 
dt4 = dt4.drop(['Potential Price', 'Trade Potential'], axis = 1) 


dt3.rename(columns = {'In Trade Potential Price':'Potential Price'}, inplace = True) 
dt4.rename(columns = {'In Trade Potential Price':'Potential Price'}, inplace = True)

dt3.rename(columns = {'RRR Potential in Trade':'RRR Potential'}, inplace = True) 
dt4.rename(columns = {'RRR Potential in Trade':'RRR Potential'}, inplace = True)

#3)
dt = dt1.append(dt2,ignore_index=True)
dt = dt.append(dt3,ignore_index=True)
dt = dt.append(dt4,ignore_index=True)

dt.sample(5)



,Instrument,No,Entry Date,Entry Time,Exit Time,Comissions,Gain/Loss,Quantity,Setup,Buy/Sell,...,Exit 3,lvl 2,Performance Grade,Pivot Description,SandR,Dilution,Highest Price2,Shares,Position Size,Time2
51,SNAP,52,2019-02-13 00:00:00,09:59:00,10:04:00,2.56,-22.11,250.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,CCCL,16,2019-01-04 00:00:00,10:10:00,10:29:00,5.07,-43.45,500.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,BPTH,89,2019-03-06 00:00:00,10:12:00,10:30:00,2.02,30.36,66.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,CLRB,47,2019-05-15 00:00:00,10:48:00,10:53:00,10.00,173.74,1250.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,VXRT,4,2019-03-19 00:00:00,09:48:00,10:01:00,2.54,87.50,250.0,BO,BUY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filtering:



1) Choose columns xyz because ......

2) Select only long pattern breakout setups 

3) Drop only long pattern breakout setups, due to their irrelevance

4) round all numerical vlaues to maximum 2 numbers after the decimal point. 



In [5]:
#1)
dt = dt[["Instrument", "Entry Date","Entry Time","Exit Time","Setup","Buy/Sell","Intended Entry","Entry Price"
,"SL Price", "Exit Price", "Highest Price", "Potential Price","Volume Exit","Wick Exit","Price Behaviour","Sector"
, "Catalyst", "Pattern", "Float","RRR in-trade","Negative RRR in-trade","Time","Outcome","Missed RRR on Entry"
,"Missed RRR","RRR Potential","Hard RRR Potential","RRR Difference","RRR Realized","RRR Volume Exit","RRR Wick Exit"
,"RRR Joint Wick and Volume Exit","Table Number"]]

#2)
setups = ['BO','BOT','VBO','FPH','VF']

dt = dt[(dt['Buy/Sell'] == "BUY") & (dt['Setup'].isin(setups))].reset_index(drop=True)

#3)

# Buy/Sell column is now irrelevant because all positions in this dataset are Buy
# Setup column is also irrelevant because all setups are Break Outs or a subgroup of a Break Out trade.
dt =  dt.drop(['Buy/Sell','Setup'], axis =1)

num_colums = ["Intended Entry","Entry Price","SL Price", "Exit Price", "Highest Price", "Potential Price","Volume Exit"
,"Wick Exit","Price Behaviour", "Float","RRR in-trade","Negative RRR in-trade","Time","Missed RRR on Entry","Missed RRR"
,"RRR Potential","Hard RRR Potential","RRR Difference","RRR Realized","RRR Volume Exit","RRR Wick Exit"
,"RRR Joint Wick and Volume Exit","Table Number"]

#4)
def roundUp (x):
    if isinstance(x, float):
        x = np.around(x,2)
        s = str(x)
        s.rstrip('0').rstrip('.') if '.' in s else s
        x = float(s)
    return x

for col in num_colums:
    dt[col] = dt[col].apply(lambda x: roundUp(x))

In [6]:
dt


,Instrument,Entry Date,Entry Time,Exit Time,Intended Entry,Entry Price,SL Price,Exit Price,Highest Price,Potential Price,...,Missed RRR on Entry,Missed RRR,RRR Potential,Hard RRR Potential,RRR Difference,RRR Realized,RRR Volume Exit,RRR Wick Exit,RRR Joint Wick and Volume Exit,Table Number
0,YECO,2018-12-13 00:00:00,09:56:00,09:59:00,5.30,5.32,5.20,5.60,5.60,6.58,...,0.20,0.20,12.60,12.80,9.80,2.80,6.80,2.80,6.80,0
1,YECO,2018-12-13 00:00:00,10:11:00,10:11:00,6.12,6.11,6.02,6.42,6.42,6.58,...,-0.10,-0.10,4.70,4.60,1.60,3.10,2.90,3.10,2.90,0
2,ALQA,2018-12-14 00:00:00,10:21:00,10:29:00,3.26,3.25,3.21,3.16,3.31,3.31,...,-0.20,0.80,1.20,1.00,3.00,-1.80,-1.80,-1.80,-1.80,0
3,INPX,2018-12-20 00:00:00,10:21:00,10:21:00,4.92,4.92,4.82,5.22,5.22,5.93,...,0.00,0.00,10.10,10.10,7.10,3.00,6.80,3.00,6.80,0
4,DOVA,2018-12-21 00:00:00,10:42:00,11:01:00,6.75,6.77,6.69,6.93,6.93,6.98,...,0.33,0.33,3.50,3.83,0.83,2.67,-0.50,2.67,-0.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,OBLN,2019-05-30 00:00:00,10:06:00,10:11:00,1.62,1.62,1.54,1.53,1.62,1.62,...,-0.03,-0.03,0.02,0.00,1.10,-1.08,-1.08,-1.08,-1.08,2
146,DPW,2019-12-11 00:00:00,10:20:00,11:21:00,1.15,1.15,1.06,1.37,1.70,1.70,...,0.00,0.00,6.11,6.11,3.67,2.44,4.56,2.44,4.56,3
147,ROYT,2019-12-13 00:00:00,10:15:00,10:50:00,0.53,0.53,0.50,0.62,0.70,0.70,...,0.00,0.00,5.67,5.67,2.64,3.02,4.00,3.02,4.00,3
148,POAI,2020-02-05 00:00:00,10:00:00,10:04:00,4.75,4.75,4.30,4.30,4.79,4.79,...,0.00,0.00,0.09,0.09,1.09,-1.00,-1.00,-1.00,-1.00,3


In [15]:
dt['SPY Week Change'] = dt['Entry Date']
dt['SPY Month Change'] = dt['Entry Date']
dt['SPY 3 Month Change'] = dt['Entry Date']

SP500 = fdata.financialData('SPY')

week = 7
month = 30
three_months = 90



dt['SPY Week Change'] = dt['Entry Date'].apply(lambda x: SP500.percentChange(x.date(), week) )
dt['SPY Month Change'] = dt['Entry Date'].apply(lambda x: SP500.percentChange(x.date(), month) )
dt['SPY 3 Month Change'] = dt['Entry Date'].apply(lambda x: SP500.percentChange(x.date(), three_months) )

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [16]:
dt.to_excel("data_clean.xlsx")  